# Energy Reconstruction Using CNN - Both Charges and Cos(Zenith)

In [1]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt

from datetime import datetime

from tensorflow import keras
from keras import layers
from keras import models
from keras import callbacks

#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Conv2D, Flatten
#from tensorflow.keras.callbacks import ModelCheckpoint

from data_tools import load_preprocessed, dataPrep, nameModel

simPrefix = os.getcwd()+'\\simdata'

In [2]:
tf.__version__

'2.6.0'

## Model Design

In [3]:
# Name for model
name = 'baseline'
i = 0
while(os.path.exists('untrainedModels/{}.h5'.format(name+str(i)))):
    i = i + 1
name = name+str(i)

# Data preparation: no merging of charge (q), no time layers included (t=False), data normalized from 0-1
prep = {'q':None, 't':False, 'normed':True, 'reco':'plane', 'cosz':False}
print(name)

baseline0


In [4]:
# Create model using functional API for multiple inputs
charge_input=keras.Input(shape=(10,10,2,))

conv1_layer = layers.Conv2D(64,kernel_size=3,padding='same',activation='relu')(charge_input)

conv2_layer = layers.Conv2D(32,kernel_size=3,padding='same',activation='relu')(conv1_layer)

conv3_layer = layers.Conv2D(16, kernel_size=3, padding='same',activation="relu")(conv2_layer)

flat_layer = layers.Flatten()(conv3_layer)
zenith_input=keras.Input(shape=(1,))
concat_layer = layers.Concatenate()([flat_layer,zenith_input])

dense1_layer = layers.Dense(256,activation='relu')(concat_layer)

dense2_layer = layers.Dense(256,activation='relu')(dense1_layer)

dense3_layer = layers.Dense(256,activation="relu")(dense2_layer)

output = layers.Dense(1)(dense3_layer)

model = models.Model(inputs=[charge_input,zenith_input],outputs=output,name=name)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])

In [5]:
model.summary()

Model: "baseline0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 10, 2)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 10, 10, 64)   1216        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 10, 10, 32)   18464       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 10, 10, 16)   4624        conv2d_1[0][0]                   
__________________________________________________________________________________________

In [6]:
model.save('untrainedModels/%s.h5'% name)
np.save('untrainedModels/%s.npy' % name,prep)